In [1]:
# Install required libraries
!pip install transformers sentencepiece sacrebleu indic-nlp-library --quiet

# Mount Google Drive (to store datasets and model checkpoints)
"""from google.colab import drive
drive.mount('/content/drive')"""

# Check if GPU is available
"""import torch
if torch.cuda.is_available():
    print("GPU is available and ready!")
else:
    print("Please enable GPU in Runtime settings.")"""


'import torch\nif torch.cuda.is_available():\n    print("GPU is available and ready!")\nelse:\n    print("Please enable GPU in Runtime settings.")'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq


In [3]:
if torch.cuda.is_available():
    print("GPU is available and ready!")
else:
    print("Please enable GPU in Runtime settings.")

GPU is available and ready!


In [4]:
import os

# Define dataset path
dataset_path = "Tamil-English-Dataset-master"  # Adjust the path if needed

# List all files in the dataset directory
dataset_files = os.listdir(dataset_path)

print("Files in Dataset Directory:")
print(dataset_files)


Files in Dataset Directory:
['Dataset', 'README.md']


In [5]:
# Define the Dataset folder path
dataset_folder_path = os.path.join(dataset_path, "Dataset")

# List all files in the Dataset folder
dataset_folder_files = os.listdir(dataset_folder_path)

print("Files in the 'Dataset' Folder:")
print(dataset_folder_files)


Files in the 'Dataset' Folder:
['data.en1', 'data.en2', 'data.en3', 'data.en4', 'data.en5', 'data.en6', 'data.ta1', 'data.ta2', 'data.ta3', 'data.ta4', 'data.ta5', 'data.ta6']


In [6]:
# Function to display the first few lines of a file
def read_file_sample(file_path, num_lines=5):
    print(f"Reading {file_path}:")
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= num_lines:
                break
            print(line.strip())

# Paths to Tamil and English files
tamil_file = os.path.join(dataset_folder_path, "data.ta1")  # Example file
english_file = os.path.join(dataset_folder_path, "data.en1")  # Corresponding file

# Read sample lines
read_file_sample(tamil_file)
read_file_sample(english_file)


Reading Tamil-English-Dataset-master\Dataset\data.ta1:
ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தினால் எறிந்துபோட்ட சகல பணிமுட்டுகளையும் முஸ்திப்பாக்கிப் பரிசுத்தம்பண்ணினோம்; இதோ , அவைகள் கர்த்தரின் ஆலயத்திற்கு முன்பாக இருக்கிறது என்றார்கள் .
சர்வதேச நாணய நிதியம் இலங்கைக்கு கடன் வழங்கினால் இதே போன்ற நிபந்தனைகள் திணிக்கப்படும் .
தற்போது அதற்கு எதிராக வாதாடுகிறார் சர்வதேச சட்டத்தை செயல்படுத்துவதற்குப் பதிலாக புதிய சட்டம் உருவாக்கப்பட்டு நிறுவப்பட வேண்டும் என்று எழுதுகிறார் .
அமெரிக்காவின் மூன்றாம் பெரிய கார் தயாரிப்பு நிறுவனமான கிறைஸ்லர் வியாழனன்று நியூ யோர்க்கில் திவாலடைந்ததற்காக மனு செய்தது; அத்தியாயம் 11 ன் படி மறு சீரமைத்து வெளிவரும் வரை அது தன்னுடைய உற்பத்தி நிலையங்களை மூடும் என்றும் அறிவித்துள்ளது .
மேலும் இனைவிட்டு தலிபானால் வெளியேற்றப்பட்ட 1995 இல் இருந்து ஈரானில் கூடுதலாக வாழ்ந்துவந்துள்ளார் .
Reading Tamil-English-Dataset-master\Dataset\data.en1:
moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , 

In [7]:
# Function to merge files
def merge_files(input_files, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for file in input_files:
            with open(file, 'r', encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line.strip() + '\n')

# Paths for merged files
merged_tamil_file = os.path.join(dataset_folder_path, "merged_tamil.txt")
merged_english_file = os.path.join(dataset_folder_path, "merged_english.txt")

# Merge all Tamil and English files
tamil_files = [os.path.join(dataset_folder_path, f) for f in dataset_folder_files if f.startswith("data.ta")]
english_files = [os.path.join(dataset_folder_path, f) for f in dataset_folder_files if f.startswith("data.en")]

merge_files(tamil_files, merged_tamil_file)
merge_files(english_files, merged_english_file)

print(f"Tamil sentences merged into: {merged_tamil_file}")
print(f"English sentences merged into: {merged_english_file}")


Tamil sentences merged into: Tamil-English-Dataset-master\Dataset\merged_tamil.txt
English sentences merged into: Tamil-English-Dataset-master\Dataset\merged_english.txt


In [8]:
# Count the number of lines in Tamil and English merged files
tamil_line_count = sum(1 for line in open(merged_tamil_file, 'r', encoding='utf-8'))
english_line_count = sum(1 for line in open(merged_english_file, 'r', encoding='utf-8'))

print(f"Number of lines in Tamil file: {tamil_line_count}")
print(f"Number of lines in English file: {english_line_count}")


Number of lines in Tamil file: 289451
Number of lines in English file: 289451


In [9]:
# Paths for cleaned aligned files
aligned_tamil_file = "Tamil-English-Dataset-master/Dataset/aligned_tamil.txt"
aligned_english_file = "Tamil-English-Dataset-master/Dataset/aligned_english.txt"

# Align Tamil and English files by truncating extra lines
def align_files(tamil_path, english_path, output_tamil, output_english):
    with open(tamil_path, 'r', encoding='utf-8') as tamil_file, \
         open(english_path, 'r', encoding='utf-8') as english_file:
        tamil_lines = tamil_file.readlines()
        english_lines = english_file.readlines()

    # Ensure both files have the same number of lines
    min_lines = min(len(tamil_lines), len(english_lines))
    aligned_tamil_lines = tamil_lines[:min_lines]
    aligned_english_lines = english_lines[:min_lines]

    # Write aligned lines to new files
    with open(output_tamil, 'w', encoding='utf-8') as tamil_out, \
         open(output_english, 'w', encoding='utf-8') as english_out:
        tamil_out.writelines(aligned_tamil_lines)
        english_out.writelines(aligned_english_lines)

    print(f"Aligned Tamil file saved to: {output_tamil}")
    print(f"Aligned English file saved to: {output_english}")
    print(f"Number of aligned sentences: {min_lines}")

# Align the files
align_files(merged_tamil_file, merged_english_file, aligned_tamil_file, aligned_english_file)


Aligned Tamil file saved to: Tamil-English-Dataset-master/Dataset/aligned_tamil.txt
Aligned English file saved to: Tamil-English-Dataset-master/Dataset/aligned_english.txt
Number of aligned sentences: 289451


In [11]:
import random

# Paths for aligned files
aligned_tamil_file = "Tamil-English-Dataset-master/Dataset/aligned_tamil.txt"
aligned_english_file = "Tamil-English-Dataset-master/Dataset/aligned_english.txt"

# Output paths for split files
train_tamil_file = "Tamil-English-Dataset-master/Dataset/train_tamil.txt"
train_english_file = "Tamil-English-Dataset-master/Dataset/train_english.txt"
val_tamil_file = "Tamil-English-Dataset-master/Dataset/val_tamil.txt"
val_english_file = "Tamil-English-Dataset-master/Dataset/val_english.txt"
test_tamil_file = "Tamil-English-Dataset-master/Dataset/test_tamil.txt"
test_english_file = "Tamil-English-Dataset-master/Dataset/test_english.txt"

# Function to normalize text
def normalize_text(lines):
    normalized = []
    for line in lines:
        line = line.strip()  # Remove extra spaces
        line = line.replace("“", '"').replace("”", '"')  # Standardize quotes
        line = line.replace("‘", "'").replace("’", "'")  # Standardize apostrophes
        normalized.append(line)
    return normalized

# Read and normalize aligned data
with open(aligned_tamil_file, 'r', encoding='utf-8') as tamil_file, \
     open(aligned_english_file, 'r', encoding='utf-8') as english_file:
    tamil_lines = normalize_text(tamil_file.readlines())
    english_lines = normalize_text(english_file.readlines())

# Shuffle and split data
data = list(zip(tamil_lines, english_lines))
random.shuffle(data)

train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))

train_data = data[:train_size]
val_data = data[train_size:train_size + val_size]
test_data = data[train_size + val_size:]

# Write split data to files
def write_data(file_path, data, index):
    with open(file_path, 'w', encoding='utf-8') as file:
        for pair in data:
            file.write(pair[index] + '\n')

write_data(train_tamil_file, train_data, 0)
write_data(train_english_file, train_data, 1)
write_data(val_tamil_file, val_data, 0)
write_data(val_english_file, val_data, 1)
write_data(test_tamil_file, test_data, 0)
write_data(test_english_file, test_data, 1)

print("Data split completed!")
print(f"Training Tamil: {train_tamil_file}")
print(f"Training English: {train_english_file}")
print(f"Validation Tamil: {val_tamil_file}")
print(f"Validation English: {val_english_file}")
print(f"Test Tamil: {test_tamil_file}")
print(f"Test English: {test_english_file}")


Data split completed!
Training Tamil: Tamil-English-Dataset-master/Dataset/train_tamil.txt
Training English: Tamil-English-Dataset-master/Dataset/train_english.txt
Validation Tamil: Tamil-English-Dataset-master/Dataset/val_tamil.txt
Validation English: Tamil-English-Dataset-master/Dataset/val_english.txt
Test Tamil: Tamil-English-Dataset-master/Dataset/test_tamil.txt
Test English: Tamil-English-Dataset-master/Dataset/test_english.txt


In [15]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Load mBART model and tokenizer
model_name = "facebook/mbart-large-50-many-to-one-mmt"
tokenizer = MBart50Tokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

print("mBART Model and Tokenizer loaded successfully!")


pytorch_model.bin:  42%|####1     | 1.02G/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

mBART Model and Tokenizer loaded successfully!


In [ ]:
"""from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load NLLB model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("NLLB Model and Tokenizer loaded successfully!")
"""

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

NLLB Model and Tokenizer loaded successfully!


In [16]:
# List all supported language codes for mBART50
print("Supported Language Codes:")
print(tokenizer.lang_code_to_id)


Supported Language Codes:
{'ar_AR': 250001, 'cs_CZ': 250002, 'de_DE': 250003, 'en_XX': 250004, 'es_XX': 250005, 'et_EE': 250006, 'fi_FI': 250007, 'fr_XX': 250008, 'gu_IN': 250009, 'hi_IN': 250010, 'it_IT': 250011, 'ja_XX': 250012, 'kk_KZ': 250013, 'ko_KR': 250014, 'lt_LT': 250015, 'lv_LV': 250016, 'my_MM': 250017, 'ne_NP': 250018, 'nl_XX': 250019, 'ro_RO': 250020, 'ru_RU': 250021, 'si_LK': 250022, 'tr_TR': 250023, 'vi_VN': 250024, 'zh_CN': 250025, 'af_ZA': 250026, 'az_AZ': 250027, 'bn_IN': 250028, 'fa_IR': 250029, 'he_IL': 250030, 'hr_HR': 250031, 'id_ID': 250032, 'ka_GE': 250033, 'km_KH': 250034, 'mk_MK': 250035, 'ml_IN': 250036, 'mn_MN': 250037, 'mr_IN': 250038, 'pl_PL': 250039, 'ps_AF': 250040, 'pt_XX': 250041, 'sv_SE': 250042, 'sw_KE': 250043, 'ta_IN': 250044, 'te_IN': 250045, 'th_TH': 250046, 'tl_XX': 250047, 'uk_UA': 250048, 'ur_PK': 250049, 'xh_ZA': 250050, 'gl_ES': 250051, 'sl_SI': 250052}


In [17]:
# Set the source and target languages for mBART
tokenizer.src_lang = "ta_IN"  # Tamil
tokenizer.tgt_lang = "en_XX"  # English


In [20]:

# Paths to split files
train_tamil_file = "Tamil-English-Dataset-master/Dataset/train_tamil.txt"
train_english_file = "Tamil-English-Dataset-master/Dataset/train_english.txt"
val_tamil_file = "Tamil-English-Dataset-master/Dataset/val_tamil.txt"
val_english_file = "Tamil-English-Dataset-master/Dataset/val_english.txt"

In [21]:


# Function to tokenize data
def tokenize_data(tamil_file, english_file):
    with open(tamil_file, 'r', encoding='utf-8') as ta, open(english_file, 'r', encoding='utf-8') as en:
        tamil_lines = ta.readlines()
        english_lines = en.readlines()

    # Tokenize line by line
    inputs = tokenizer(tamil_lines, max_length=128, truncation=True, padding=True, return_tensors="pt")
    targets = tokenizer(english_lines, max_length=128, truncation=True, padding=True, return_tensors="pt")
    return inputs, targets

# Tokenize training and validation data
train_inputs, train_targets = tokenize_data(train_tamil_file, train_english_file)
val_inputs, val_targets = tokenize_data(val_tamil_file, val_english_file)

# Prepare datasets
train_dataset = Dataset.from_dict({
    "input_ids": train_inputs["input_ids"],
    "attention_mask": train_inputs["attention_mask"],
    "labels": train_targets["input_ids"]
})

val_dataset = Dataset.from_dict({
    "input_ids": val_inputs["input_ids"],
    "attention_mask": val_inputs["attention_mask"],
    "labels": val_targets["input_ids"]
})

print("Datasets prepared successfully!")


Datasets prepared successfully!


In [22]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import os

In [ ]:


# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Define data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="Translator/fine_tuned_model",  # Path to save the model
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Lower batch size to avoid memory issues
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Simulate larger batch size
    weight_decay=0.01,
    save_total_limit=3,  # Keep only the 3 most recent checkpoints
    num_train_epochs=3,
    predict_with_generate=True,  # Enable sequence generation during evaluation
    fp16=True,  # Enable mixed precision for faster training
    logging_dir="Translator/logs",  # Log directory
    logging_steps=500,
    optim="adafactor",  # Use memory-efficient optimizer
    dataloader_pin_memory=False  # Avoid GPU memory over-allocation
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,  # Safe to use in current version
    data_collator=data_collator
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("Translator/fine_tuned_model")
print("Model fine-tuned and saved successfully!")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
c:\Users\dhanu\.conda\envs\idk_gpu\lib\site-packages\accelerate\accelerator.py:479: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/43416 [00:00<?, ?it/s]

KeyboardInterrupt: 

: 

In [ ]:
from transformers import Seq2SeqTrainer

# Load the model from the fine-tuned path
model = model.from_pretrained("Translator/fine_tuned_model")

# Initialize the trainer with test data
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    eval_dataset= test_data,  # Use the test dataset for evaluation
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results
print(f"Test Results: {eval_results}")


In [ ]:
# Example Tamil sentences for translation
example_sentences = [
    "வணக்கம், எப்படி இருக்கின்றீர்கள்?",
    "இந்த புத்தகம் மிகவும் ஆரோக்கியமானது.",
    "இந்தப் படம் மிக அழகானது."
]

# Translate the sentences
translations = []
for sentence in example_sentences:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(inputs["input_ids"])
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(translation)

# Print the translations
for i, translation in enumerate(translations):
    print(f"Original: {example_sentences[i]}")
    print(f"Translation: {translation}")
    print("-" * 50)


In [ ]:
# Save the model and tokenizer to a directory
model.save_pretrained("Translator/final_model")
tokenizer.save_pretrained("Translator/final_model")

# Optionally save the training arguments if you'd like to resume training later
training_args.save_to_json("Translator/final_model/training_args.json")


In [ ]:
# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("Translator/final_model")
tokenizer = AutoTokenizer.from_pretrained("Translator/final_model")

# Example Tamil sentences for translation
example_sentences = [
    "இன்று மிக அழகான நாள்!",
    "நான் தமிழ் பேச விரும்புகிறேன்.",
    "உங்களுடன் சந்திப்பதில் மகிழ்ச்சி."
]

# Translate the sentences
translations = []
for sentence in example_sentences:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(inputs["input_ids"])
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(translation)

# Print the translations
for i, translation in enumerate(translations):
    print(f"Original: {example_sentences[i]}")
    print(f"Translation: {translation}")
    print("-" * 50)
